# Setup

First things first, let's check our working directory.

In [ ]:
pwd  # some bash commands will work without additional characters in Jupyter notebooks if automagic is on

To build our pipeline, we'll need to make sure we have access to some dependencies first.  All of these are included in the environment.yaml file used to build the binder container, but you can use conda to install them if you're working outside of the binder container (`conda install -c bioconda <tool=version>`).  Let's check the versions of the dependencies to ensure they're there and accessible.

In [ ]:
%%bash  # to run a whole cell in bash
echo -e "snakemake:  $(snakemake --version)"
echo -e "samtools:   $(samtools --version | head -n1)"
echo -e "picard:     $(picard --version)"
echo -e "bwa:        $(bwa |& grep Version)"
echo -e "gatk:       $(gatk --version |& grep 'The Genome Analysis Toolkit')"
echo -e "python:     $(python3 --version |& grep Python)"

# Input data

We're going to start with FASTQ files from real human samples, but instead of whole genome data, we're focusing on chromosome 5, position 100000000-101500000.  Our reference genome file also only includes a portion of chr5.  This is to keep file sizes small and run times short!  Now, let's make sure we can see the raw files we intend to work on.

In [ ]:
ls -lh raw_data/

In [ ]:
# what does a fastq look like?
!head -n8 raw_data/Patient_A.r1.fastq  # to run a single line of bash, prepend with "!"

In [ ]:
ls -lh ref_genome/

In [ ]:
# what does a reference genome look like?
!head -n4 ref_genome/chr5_ref.fasta

# Plan

__Goal:__ assemble a working DNA-seq pipeline!

- Align sequencing data to a reference genome
- Call variants in the aligned data
- Annotate variants

# First rule: indexing

Our first rule is going to take our reference genome file, and index it so that the alignment tool can read it.  We can write the rule in any text editor, but for this class, we'll write it here in the notebook and save it to a file.

In [ ]:
%%writefile snakefile_test1

ref = 'ref_genome/chr5_ref.fasta'
tempRef = os.path.splitext(ref)[0]
if ref.endswith(('.gz')):
    refNoExt = os.path.splitext(tempRef)[0]
else:
    refNoExt = tempRef

rule index_ref:
    '''
    This rule creates the indices needed by
    bwa in order to use a reference fasta,
    and by GATK to call variants.
    '''
    input:
        ref
    output:
        o1 = ref + '.amb',
        o2 = ref + '.ann',
        o3 = ref + '.bwt',
        o4 = ref + '.pac',
        o5 = ref + '.sa',
        o6 = ref + '.fai',
        o7 = refNoExt + '.dict'
    shell:
        'bwa index {input};'
        'samtools faidx {input};'
        'picard CreateSequenceDictionary REFERENCE={input} OUTPUT={output.o7}'

Let's test our first rule!  You can run this rule from the command line or here in the notebook using cell magic.  For our first test, we're going to try a __dry-run__ by using the `-n` flag.  If your snakefile is called something other than "Snakefile," use `-s <filename>`.

In [ ]:
!snakemake -ns snakefile_test1

Looks great!  Note that the dry-run does not actually execute any jobs; it shows the execution plan.  

Now let's try running our pipeline for real.  Add the `-p` flag to print the job that's run for each rule.

In [ ]:
!snakemake -ps snakefile_test1

Hooray!!  Our first rule worked.  Note that Snakemake stdout provides a beautiful log of steps run and errors encountered.

What happens if we try to run it again?

In [ ]:
!snakemake -ps snakefile_test1

# Second rule: alignment

In [ ]:
# include trimming?

Our next rule is going to take the short reads in our FASTQ files, and align them to a reference sequence using a tool called [__bwa__](https://github.com/lh3/bwa).  

In [ ]:
rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = 'raw_data/Patient_A.r1.fastq',
        fq2 = 'raw_data/Patient_A.r2.fastq'
    output:
        'aligned/PatientA.bam'
    shell:
        'bwa mem {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

Note how not all the listed input files are actually needed in the command line, but they are required for the command to run successfully (i.e. bwa will give an error if the index files are not there).  Snakemake recommends that you include all file dependencies in the input section, even if they're not used in the command invocation.

Hold up.  We don't want to hard-code our sample files into a pipeline, or else we have to change the code for every sample! How do we handle this?

![xkcd](https://imgs.xkcd.com/comics/the_general_problem.png)

In [ ]:
SAMPLES = ['Patient_A.r1', 'Patient_A.r2', 'Patient_B.r1', 'Patient_B.r2',]

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq = 'raw_data/{sample}.fastq'
    output:
        'aligned/{sample}.bam'
    shell:
        'bwa mem {input.ref} {input.fq} | samtools sort -o {output}'

What if we use a list, as above?  This would run an alignment on each fastq individually, which would be fine if we had single-end reads.  But, we have paired-end reads, which means you've sequenced in both directions, and you need to align two related fastq files per sample.

In [ ]:
rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    shell:
        'bwa mem {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

What if we use a dict?  Much better!  Now our `rule align_fastqs` is generalizable.  If you were using this pipeline in real life, you'd probably require the user to provide a sample file where each line has the sample name, fastq1, and fastq2, and you'd read that in to a dict (rather than explicitly defining a dict like we did).

Note that input (or params) can be the return value of a function, as in this example.

Let's put the two rules together, and then try running them.

In [ ]:
%%writefile snakefile_test2

ref = 'ref_genome/chr5_ref.fasta'
rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule index_ref:
    '''
    This rule creates the indices needed by
    bwa in order to use a reference fasta.
    '''
    input:
        ref
    output:
        ref + '.amb',
        ref + '.ann',
        ref + '.bwt',
        ref + '.pac',
        ref + '.sa'
    shell:
        'bwa index {input}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    shell:
        'bwa mem {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

In [ ]:
!snakemake -ps snakefile_test2

Oh no!  What went wrong?  We haven't given snakemake a target file.  Let's add a `rule all`.

In [ ]:
%%writefile snakefile_test3

ref = 'ref_genome/chr5_ref.fasta'
rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('aligned/{sample}.bam', sample=sampleDict.keys())

rule index_ref:
    '''
    This rule creates the indices needed by
    bwa in order to use a reference fasta.
    '''
    input:
        ref
    output:
        ref + '.amb',
        ref + '.ann',
        ref + '.bwt',
        ref + '.pac',
        ref + '.sa'
    shell:
        'bwa index {input}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    shell:
        'bwa mem {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'

Now snakemake knows that `{sample}` should expand to PatientA and PatientB, and that the pipeline should end up producing the files `'aligned/{sample}.bam'`.  Let's try running it (this will take a minute or two):

In [ ]:
!snakemake -ps snakefile_test3

Yay!  A few things of note:
- We now have one aligned bam file per patient!
- Snakemake automatically created the directory `aligned/` for us.
- The stdout also includes a bunch of information from bwa.  There are ways to clean this up, but we're going to skip over that for now.
- Snakemake saw that the indexed reference files were already created, so it did not re-run that rule.

# Third rule: calling

This third rule will compare our aligned sequences to the reference genome, look at places where there's a discrepancy, and report back those variants.  We'll need to write the rule and update the rule all with the new target file.

In [ ]:
%%writefile snakefile_test4

# need a fasta fai for ref

ref = 'ref_genome/chr5_ref.fasta'
rawDataPath = 'raw_data/'
sampleDict = {
    'PatientA':['Patient_A.r1.fastq', 'Patient_A.r2.fastq'],
    'PatientB':['Patient_B.r1.fastq', 'Patient_B.r2.fastq']
}

def get_read1_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read1

def get_read2_fastq(wildcards):
    (read1, read2) = sampleDict[wildcards.sample]
    return rawDataPath + read2


rule all:
    input:
        expand('called/{sample}.vcf', sample=sampleDict.keys())

rule index_ref:
    '''
    This rule creates the indices needed by
    bwa in order to use a reference fasta.
    '''
    input:
        ref
    output:
        ref + '.amb',
        ref + '.ann',
        ref + '.bwt',
        ref + '.pac',
        ref + '.sa'
    shell:
        'bwa index {input}'

rule align_fastqs: 
    input:
        ref = ref,
        r1 = ref + '.amb',
        r2 = ref + '.ann',
        r3 = ref + '.bwt',
        r4 = ref + '.pac',
        r5 = ref + '.sa',
        fq1 = get_read1_fastq,
        fq2 = get_read2_fastq
    output:
        'aligned/{sample}.bam'
    shell:
        'bwa mem {input.ref} {input.fq1} {input.fq2} | samtools sort -o {output}'
        
rule call_variants:
    input:
        ref = ref,
        bam = 'aligned/{sample}.bam'
    output:
        'called/{sample}.vcf'
    shell:
        'gatk HaplotypeCaller -I:{input.bam} -O:{output} -R:{input.ref}'

In [ ]:
!snakemake -ps snakefile_test4